### Environment

In [1]:
import openai
import os
import json
import torch
import numpy as np
import pandas as pd

In [58]:
os.environ["OPENAI_API_KEY"] = "fk223678-Epq30sDYIn8TKUnTogD8C0Lu5QEF5bwx"
os.environ["OPENAI_API_BASE"] = "https://oa.api2d.net/v1"

### RAG

##### Loading

In [3]:
from langchain_community.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader("/gemini/data-1/初赛训练数据集.pdf")
data=loader.load()

In [5]:
data_list=[]
for i in range (0,len(data)):
  page=data[i].metadata["page"]
  data[i].metadata["page"]=page+1
  if i>=2 and i<=6:
    data[i].metadata["category"]="catalog"
  else:
    data[i].metadata["category"]="content"
  if data[i].page_content!="":
    data_list.append(data[i])

我现在估计需要进行一下这边的预处理的工作。
这边基本上都是没有加上任何的meta的。估计之后需要加上meta.
注意这边的meta信息是不对的，因为这边的meta信息page是从0开始的，但是page的信息需要从1开始。所以这边有这样的一个page的更改。



这边我估计就是直接放弃所有的split的过程

##### Indexing

In [3]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [4]:
from langchain.vectorstores import Chroma
vectordb = Chroma(persist_directory="/gemini/code/vectordb", embedding_function=embeddings)

In [5]:
vectordb._collection.count()

334

#### 构建的retriever_k

In [41]:
retriever_k =vectordb.as_retriever(search_kwargs={"k": 3})

#### Multiple Querying

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,
    model_name="gpt-3.5-turbo-1106",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
multiple_qa_system_prompt = """你是一个汽车维修和汽车销售的专家，您的任务是将下列提问改写为5个含义相近但当不相同的句子。这些句子有不同的表现形式，但都是围绕着提问的主题。请提供这些替代问题，并用换行符分隔。"""
multiple_qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", multiple_qa_system_prompt),
        ("human", "{question}"),
    ]
 )

这边还有一个是AI方向的。

In [9]:
multiple_qa_chain = (multiple_qa_prompt| llm)

In [10]:
def create_original_query(query):
    multiple_qa_system_prompt = """你是一个汽车维修和汽车销售的专家，您的任务是将下列提问改写为5个含义相近但当不相同的句子。这些句子有不同的表现形式，但都是围绕着提问的主题。请提供这些替代问题，并用换行符分隔。"""
    multiple_qa_prompt = ChatPromptTemplate.from_messages([
        ("system", multiple_qa_system_prompt),
        ("human", "{question}"),
    ])
    multiple_qa_chain = (multiple_qa_prompt| llm)
    question_string = multiple_qa_chain.invoke({"question": query})
    queries=question_string.content.splitlines()
    return queries

感觉还行

#### Cross encoding

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
model = AutoModelForSequenceClassification.from_pretrained("/gemini/pretrain")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("/gemini/pretrain")

In [42]:
def create_documents(query):
    doc_list=[]
    page_list=[]
    query_list=[]

    for q in queries:
        results=retriever_k.get_relevant_documents(q)
        doc_strings = [doc.page_content for doc in results]
        doc_pages=[doc.metadata["page"] for doc in results]
        doc_s=list(np.unique(doc_strings))
        doc_p=list(np.unique(doc_pages))
        query_list.append([q]*len(doc_s))
        doc_list.append(doc_s)
        page_list.append(doc_p)

    pairs=[]
    for i in range(len(doc_list)):
        docs=doc_list[i]
        query_=query_list[i]
        for j in range(len(docs)):
            pairs.append([query_[j],docs[j]])
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt').to("cuda")
        score_k = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores=score_k.cpu().numpy()
    df=pd.DataFrame([np.array(doc_list).flatten(),np.array(page_list).flatten(),scores]).T
    df.columns=["text","page","score"]
    df.drop_duplicates(subset=['text'],inplace=True)
    df_sorted=df.sort_values(by='score', ascending=False)
    df_sorted=df_sorted.reset_index(drop=True)
    first_element = df_sorted.loc[0,"text"]
    first_page=df_sorted.loc[0,"page"]
    return first_element,first_page

#### retrieval qa

In [45]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,
    model_name="gpt-3.5-turbo-1106",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
)

/root/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [46]:
from langchain_core.prompts import ChatPromptTemplate
qa_system_prompt = """你是一个汽车方面的专家，请结合给定的资料，并回答最终的问题。请如实回答，如果问题在资料中找不到答案，请回答不知道。
资料：{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{question}"),
    ]
 )

In [47]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
chain = ({"context":RunnablePassthrough() , "question": RunnablePassthrough()}
    | qa_prompt
    | llm
 )

#### Testing

In [13]:
questions = json.load(open("/gemini/data-1/questions.json"))

In [59]:
for i in range(len(questions)):
  query=questions[i]["question"]
  queries=create_original_query(query)
  first_element,first_page=create_documents(queries)
  questions[i]["reference"]="page_"+str(first_page) 
  chunks=[]
  for chunk in chain.stream({"context":first_element,"question":query}):
    chunks.append(chunk.content)
  result="".join(chunks)
  questions[i]['answer'] = result

In [60]:
# Serializing json
json_object = json.dumps(questions, indent=4)

# Writing to sample.json
with open("sample_openai-bge-reranker-multiqa.json", "w") as outfile:
    outfile.write(json_object)

很神奇，就是query那边难道没有什么问题吗？会不会最终不是list?或者是还有什么多余的问题？